In [1]:
import sys
sys.path.insert(0, "/Users/aryakakodkar/Documents/qerasure/python")
sys.path.insert(1, "/Users/aryakakodkar/Documents/qerasure/build")

In [2]:
from qerasure import RotatedSurfaceCode, NoiseParams

In [3]:
code = RotatedSurfaceCode(distance=5)

In [4]:
noise = NoiseParams(p_two_qubit_erasure=0.01, p_erasure_check_error=0.05)

TypeError: validate(): incompatible function arguments. The following argument types are supported:
    1. (self: qerasure_python.NoiseParams) -> None

Invoked with: 